# Imports

In [1]:
import concurrent.futures
import io
import os
import re
import shlex
import socket
import subprocess
from pathlib import Path

import numpy as np
import tqdm

import pandas as pd
import psutil

# Parameters

In [3]:
NOTEBOOK_NAME = "generate_difficult_sudokus"

In [4]:
NOTEBOOK_PATH = Path(NOTEBOOK_NAME).resolve()
NOTEBOOK_PATH.mkdir(exist_ok=True)
NOTEBOOK_PATH

PosixPath('/data/workspace/proteinsolver/notebooks/generate_difficult_sudokus')

In [5]:
hostname = socket.gethostname()

if hostname == "strokach-cloudtop":
    NPROC = 16
elif hostname.startswith("nia") or hostname.startswith("beluga"):
    NPROC = 40
else:
    NPROC = psutil.cpu_count(logical=True)  # For some reason we see only 20 cpus on beluga 

In [ ]:
START_BATCH_IDX = int(os.getenv("START_BATCH_IDX", 10_000))
offset = int(os.getenv("SLURM_ARRAY_TASK_ID"), 0) * 10 + 5
START_BATCH_IDX += offset
START_BATCH_IDX

# Functions

In [6]:
def sudoku_is_solved(values) -> bool:
    ref = np.arange(1, 10)
    mat = values.reshape(9, 9)
    for i in range(3):
        for j in range(3):
            v = np.sort(mat[i * 3 : (i + 1) * 3, j * 3 : (j + 1) * 3], axis=None)
            if not (v == ref).all():
                return False
    for i in range(9):
        v = np.sort(mat[i, :])
        if not (v == ref).all():
            return False
    for j in range(9):
        v = np.sort(mat[:, j])
        if not (v == ref).all():
            return False
    return True


def decode_sugen_output(output):
    grid = np.empty((9, 9), dtype=np.int)
    for i, row in enumerate(output.split("\n")[:9]):
        for j, value in enumerate(row.split(" ")):
            if value == "_":
                grid[i, j] = 0
            else:
                grid[i, j] = int(value)

    info_string = output.strip().split("\n")[-1]
    try:
        difficulty = int(
            re.findall("Difficulty: (\d+)", info_string)[0]
        )
    except IndexError:
        difficulty = None
    return grid, difficulty


def generate_sudoku():
    sc = "sugen -i 5000 -t 10000 generate"
    ps = subprocess.run(shlex.split(sc), stdout=subprocess.PIPE)

    sc2 = "sugen solve"
    ps2 = subprocess.run(shlex.split(sc2), input=ps.stdout, stdout=subprocess.PIPE)

    puzzle, _ = decode_sugen_output(ps.stdout.decode())
    solution, difficulty = decode_sugen_output(ps2.stdout.decode())

    assert sudoku_is_solved(solution)
    return puzzle, solution, difficulty

In [7]:
generate_sudoku()

(array([[0, 0, 1, 0, 4, 0, 2, 0, 0],
        [0, 2, 0, 0, 0, 3, 0, 8, 0],
        [4, 0, 3, 0, 0, 0, 0, 0, 0],
        [0, 7, 6, 8, 0, 5, 0, 4, 0],
        [3, 0, 0, 4, 0, 1, 0, 0, 7],
        [0, 4, 0, 7, 0, 2, 6, 3, 0],
        [0, 0, 0, 0, 0, 0, 7, 0, 5],
        [0, 6, 0, 3, 0, 0, 0, 9, 0],
        [0, 0, 8, 0, 2, 0, 3, 0, 0]]), array([[8, 5, 1, 9, 4, 6, 2, 7, 3],
        [6, 2, 7, 1, 5, 3, 4, 8, 9],
        [4, 9, 3, 2, 7, 8, 1, 5, 6],
        [2, 7, 6, 8, 3, 5, 9, 4, 1],
        [3, 8, 9, 4, 6, 1, 5, 2, 7],
        [1, 4, 5, 7, 9, 2, 6, 3, 8],
        [9, 3, 2, 6, 8, 4, 7, 1, 5],
        [5, 6, 4, 3, 1, 7, 8, 9, 2],
        [7, 1, 8, 5, 2, 9, 3, 6, 4]]), 951)

# Generate sudokus

In [ ]:
for batch_idx in range(START_BATCH_IDX, START_BATCH_IDX + 1_000_000):
    print(f"Generating batch {batch_idx}...")
    results = []
    with concurrent.futures.ThreadPoolExecutor(NPROC) as pool:
        futures = [pool.submit(generate_sudoku) for _ in range(100_000)]
        for future in tqdm.tqdm_notebook(
            concurrent.futures.as_completed(futures), total=len(futures)
        ):
            puzzle, solution, difficulty = future.result()
            puzzle_str = "".join([str(v) for v in puzzle.reshape(-1)])
            solution_str = "".join([str(v) for v in solution.reshape(-1)])
            results.append((puzzle_str, solution_str, difficulty))
    df = pd.DataFrame(results, columns=["puzzle", "solution", "difficulty"])
    df.to_csv(NOTEBOOK_PATH.joinpath(f"sodoku_{batch_idx}.csv"), sep=",", index=False)

Generating batch 0...



Generating batch 1...
